In [4]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
import opendp.prelude as dp
dp.enable_features("contrib")
dp.enable_features("floating-point")
from utilities import *
from StabilityHist import *
from Laplace import *

In [5]:
#Level: county or ed
level = "county"

#Mechanism: laplace, stabilityhist, directencoding, randresponse, olh, hadamard, rappor
Mechanism = "laplace"
Mechanism_name = "Laplace Mechanism"

#Mechanism = "stabilityhist"
#Mechanism_name = "Stability Histogram"

#Path to data
path = "/Users/Ava/Library/CloudStorage/OneDrive-Personal/Thesis/Code/"
save = False

max_influence = 2
epsilon = np.arange(0.5,5.5, 0.5)


In [6]:
Level = level.capitalize()
size, categories, col_names = get_variables(path, level, Level)

if Mechanism == "stabilityhist":
    delta = 1/(2*size)
else:
    delta = 0 

budget = [(e, delta) for e in epsilon]

In [7]:
with open(f'commute_{level}_level_all.csv') as input_data:
    data = input_data.read()

In [8]:
#This is the dataset without differential privacy. 
histogram = (
    dp.t.make_split_dataframe(separator=",", col_names=col_names) >>
    dp.t.make_select_column(key=f"{Level}_commute", TOA=str) >>
    # Compute counts for each of the categories
    dp.t.then_count_by_categories(categories=categories)
)

sensitive_counts = histogram(data)

In [9]:
if Mechanism == "laplace":
    if level=="county":
        county_released_counts_laplace, county_elapsed_time_laplace, county_all_rmse_laplace = Laplace_Mechamism(budget, max_influence, data, histogram, sensitive_counts)
    if level == "ed":
        ed_released_counts_laplace, ed_elapsed_time_laplace, ed_all_rmse_laplace = Laplace_Mechamism(budget, max_influence, data, histogram, sensitive_counts)
if Mechanism == "stabilityhist":
    if level=="county":
        county_released_counts_stabilityhist, county_elapsed_time_stabilityhist, county_all_rmse_stabilityhist = Stability_Hist(col_names, Level, budget, max_influence, size, data, histogram, categories, sensitive_counts)
    if level == "ed":
        ed_released_counts_stabilityhist, ed_elapsed_time_stabilityhist, ed_all_rmse_stabilityhist = Stability_Hist(col_names, Level, budget, max_influence, size, data, histogram, categories, sensitive_counts)

In [10]:
len(county_released_counts_laplace[1])

1085

Save File for Analysis

In [11]:
private_dataset_df = pd.DataFrame(categories, columns = [f'{Level} Level Commute'])
private_dataset_df['True Count'] =  sensitive_counts[:-1]
for i, name in enumerate(budget):
    private_dataset_df[f'Privacy {name[0]}'] = locals()[f"{level}_released_counts_{Mechanism}"][i]

if save:
    private_dataset_df.to_csv(f'{level}_{Mechanism}_dp_df.csv', sep=',', index=False, encoding='utf-8')


In [12]:
print("Commute level counts:\n", sensitive_counts[:-1])
print("DP Commute level counts:\n", locals()[f"{level}_released_counts_{Mechanism}"][0][:-1])

Commute level counts:
 [19174, 962, 548, 132, 290, 1251, 1512, 984, 28, 12, 958, 1054, 40, 39, 9, 76, 248, 46, 12, 9, 8, 1978, 6, 1626, 52, 14, 13, 17, 3, 28, 3, 8, 6, 1, 0, 92, 218860, 15302, 22668, 19235, 3056, 41, 71, 1141, 60, 50, 796, 113, 108, 31, 140, 58, 139, 47, 32, 34, 17955, 177, 6548, 62, 369, 53, 142, 26, 518, 35, 31, 32, 33, 8, 84, 44145, 80723, 5199, 12769, 4526, 36, 57, 563, 30, 35, 818, 49, 73, 25, 67, 41, 68, 26, 18, 26, 10631, 61, 2914, 72, 125, 20, 127, 9, 172, 18, 23, 27, 16, 9, 54, 56998, 5941, 91167, 4872, 2236, 22, 29, 3470, 36, 35, 273, 47, 82, 18, 95, 43, 72, 33, 38, 59, 10995, 124, 4410, 36, 1025, 39, 124, 10, 220, 17, 32, 27, 16, 1, 45, 41779, 6154, 2150, 66937, 974, 25, 28, 233, 17, 58, 2979, 54, 62, 12, 72, 25, 84, 26, 15, 16, 6420, 69, 4835, 25, 93, 12, 62, 8, 240, 15, 9, 10, 10, 3, 1701, 18313, 11759, 3645, 3572, 80682, 320, 906, 1285, 973, 57, 1293, 71, 111, 30, 203, 135, 185, 37, 17, 30, 8673, 60, 4465, 120, 87, 21, 336, 27, 176, 34, 37, 41, 27, 4, 163

In [13]:
level

'county'

In [14]:
locals()[f"agg_{Level}_data_df"] 

KeyError: 'agg_County_data_df'